# Tutorial for implementing custom gradients in JAX

- Goal of this notebook is to better understand how to leverage acceleration libraries to implement custom derivatives.
- Although, most of the libraries like JAX, PyTorch, TensorFlow etc. can easily implement automatic differentiation, there is always a need to implement activation functions that may not be differentiable:
    - Spiking neural networks typically use LIF neurons
    - It is efficient to use simplified backward pass or truncated gradients.
    - We are now working with ternary neurons, which also have discontinuous activation.

- For these type of models, the backward pass in backpropagation or a similar algorithm has to have custom defined gradients.
- As a starting point, this notebook will go through <a href = "https://jax.readthedocs.io/en/latest/notebooks/Custom_derivative_rules_for_Python_code.html">"Custom Derivative Rules" </a> tutorial on JAX documentation page.